In [41]:
import pandas as pd

# Path to the Excel file
file_path = r'files/coor_0.xlsx'

# Reading the Excel file into a DataFrame
df_copy_copy_copy = pd.read_excel(file_path)

# Display the first few rows of the DataFrame
df_copy_copy_copy.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 84 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ball_x  586 non-null    float64
 1   ball_y  586 non-null    float64
 2   ball_t  586 non-null    float64
 3   p2_x    794 non-null    float64
 4   p2_y    794 non-null    float64
 5   p2_t    794 non-null    float64
 6   p3_x    778 non-null    float64
 7   p3_y    778 non-null    float64
 8   p3_t    778 non-null    float64
 9   p4_x    755 non-null    float64
 10  p4_y    755 non-null    float64
 11  p4_t    755 non-null    float64
 12  p5_x    785 non-null    float64
 13  p5_y    785 non-null    float64
 14  p5_t    785 non-null    float64
 15  p6_x    785 non-null    float64
 16  p6_y    785 non-null    float64
 17  p6_t    785 non-null    float64
 18  p7_x    795 non-null    float64
 19  p7_y    795 non-null    float64
 20  p7_t    795 non-null    float64
 21  p8_x    794 non-null    float64
 22  p8

This script reads the Excel file, applies linear interpolation, forward and backward fills, and then smooths the data using a rolling mean.

In [42]:
# Make a copy of the DataFrame
df_copy = df.copy()

# Define a threshold for sparse data (e.g., less than 60% of the total rows)
sparse_threshold = 0.6

# Identify columns for each player's x, y, t coordinates
players = [col.split('_')[0] for col in df_copy.columns if 'x' in col]
players = list(set(players))  # Remove duplicates

# Iterate over each player to handle sparse data and interpolation
for player in players:
    # Check the ratio of non-null values to total rows
    non_null_ratio = df_copy[f'{player}_x'].notna().sum() / len(df_copy)
    
    # If the ratio is below the threshold, mark the entire column as NaN
    if non_null_ratio < sparse_threshold:
        df_copy[[f'{player}_x', f'{player}_y', f'{player}_t']] = None

    # Ensure the columns are numeric
    df_copy[f'{player}_x'] = pd.to_numeric(df_copy[f'{player}_x'], errors='coerce')
    df_copy[f'{player}_y'] = pd.to_numeric(df_copy[f'{player}_y'], errors='coerce')

    # Interpolate for missing values within each player's coordinates
    df_copy[[f'{player}_x', f'{player}_y']] = df_copy[[f'{player}_x', f'{player}_y']].interpolate(method='linear', limit_direction='both')

    # Forward and backward fill for leading/trailing NaNs within each player's coordinates
    df_copy[[f'{player}_x', f'{player}_y']] = df_copy[[f'{player}_x', f'{player}_y']].ffill().bfill()


# Save the cleaned DataFrame back to a file if needed
df_copy.to_excel(r'files/coor_0_cleaned.xlsx', index=False)

In [43]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 84 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ball_x  800 non-null    float64
 1   ball_y  800 non-null    float64
 2   ball_t  586 non-null    float64
 3   p2_x    800 non-null    float64
 4   p2_y    800 non-null    float64
 5   p2_t    794 non-null    float64
 6   p3_x    800 non-null    float64
 7   p3_y    800 non-null    float64
 8   p3_t    778 non-null    float64
 9   p4_x    800 non-null    float64
 10  p4_y    800 non-null    float64
 11  p4_t    755 non-null    float64
 12  p5_x    800 non-null    float64
 13  p5_y    800 non-null    float64
 14  p5_t    785 non-null    float64
 15  p6_x    800 non-null    float64
 16  p6_y    800 non-null    float64
 17  p6_t    785 non-null    float64
 18  p7_x    800 non-null    float64
 19  p7_y    800 non-null    float64
 20  p7_t    795 non-null    float64
 21  p8_x    800 non-null    float64
 22  p8

In [44]:
# Total video duration in minutes
video_duration_minutes = 5

# Total number of frames
total_frames = 800

# Calculate time interval between frames
time_interval_seconds = (video_duration_minutes * 60) / (total_frames - 1)

# Generate time values for each frame
time_values = [time_interval_seconds * i for i in range(total_frames)]

# Assign time values to 't' columns in the DataFrame
for column in df_copy.columns:
    if column.endswith('_t'):
        df_copy[column] = time_values

df_copy.head()

,ball_x,ball_y,ball_t,p2_x,p2_y,p2_t,p3_x,p3_y,p3_t,p4_x,...,p27_t,p51_x,p51_y,p51_t,p105_x,p105_y,p105_t,p155_x,p155_y,p155_t
0,0.504519,0.471692,0.000000,0.302547,0.744447,0.000000,0.290804,0.606636,0.000000,0.272524,...,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000
1,0.504519,0.471692,0.375469,0.302820,0.744706,0.375469,0.290708,0.606766,0.375469,0.272608,...,0.375469,NaN,NaN,0.375469,NaN,NaN,0.375469,NaN,NaN,0.375469
2,0.504519,0.471692,0.750939,0.303129,0.745041,0.750939,0.290641,0.606881,0.750939,0.272543,...,0.750939,NaN,NaN,0.750939,NaN,NaN,0.750939,NaN,NaN,0.750939
3,0.504519,0.471692,1.126408,0.303176,0.745089,1.126408,0.290457,0.606651,1.126408,0.272388,...,1.126408,NaN,NaN,1.126408,NaN,NaN,1.126408,NaN,NaN,1.126408
4,0.504519,0.471692,1.501877,0.303144,0.744982,1.501877,0.289953,0.606263,1.501877,0.273042,...,1.501877,NaN,NaN,1.501877,NaN,NaN,1.501877,NaN,NaN,1.501877


In [47]:
import numpy as np

def calculate_distance(x_values, y_values):
    """
    Calculate the total distance traveled given x and y coordinate values.
    
    Args:
    - x_values (array-like): Array of x coordinate values.
    - y_values (array-like): Array of y coordinate values.
    
    Returns:
    - total_distance (float): Total distance traveled.
    """
    distances = np.sqrt(np.diff(x_values)**2 + np.diff(y_values)**2)
    total_distance = np.sum(distances)
    return total_distance

def calculate_top_speed(x_values, y_values, time_values):
    """
    Calculate the top speed reached by a player given x and y coordinate values and corresponding time values.
    
    Args:
    - x_values (array-like): Array of x coordinate values.
    - y_values (array-like): Array of y coordinate values.
    - time_values (array-like): Array of time values.
    
    Returns:
    - top_speed (float): Top speed reached.
    """
    distances = np.sqrt(np.diff(x_values)**2 + np.diff(y_values)**2)
    time_intervals = np.diff(time_values)
    speeds = distances / time_intervals
    top_speed = np.max(speeds)
    return top_speed

def calculate_num_touches(df, player):
    """
    Calculate the number of touches for a specific player.
    
    Args:
    - df (DataFrame): DataFrame containing player position data.
    - player (str): Name of the player.
    
    Returns:
    - num_touches (int): Number of touches for the player.
    """
    x_column = f'{player}_x'
    num_touches = df[x_column].count()
    return num_touches

def calculate_num_passes(df, player):
    """
    Calculate the number of passes for a specific player.
    
    Args:
    - df (DataFrame): DataFrame containing player position data.
    - player (str): Name of the player.
    
    Returns:
    - num_passes (int): Number of passes for the player.
    """
    x_column = f'{player}_x'
    passes = np.abs(np.diff(df[x_column])) > 0.1  # Adjust the threshold as needed
    num_passes = np.sum(passes)
    return num_passes

def calculate_num_corner_kicks(df):
    """
    Calculate the number of corner kicks.
    
    Args:
    - df (DataFrame): DataFrame containing ball position data.
    
    Returns:
    - num_corner_kicks (int): Number of corner kicks.
    """
    num_corner_kicks = df[df['ball_x'] < 0.1]['ball_x'].count()  # Adjust the threshold as needed
    return num_corner_kicks

def calculate_num_throw_ins(df):
    """
    Calculate the number of throw-ins.
    
    Args:
    - df (DataFrame): DataFrame containing ball position data.
    
    Returns:
    - num_throw_ins (int): Number of throw-ins.
    """
    num_throw_ins = df[(df['ball_x'] > 0.9) & (df['ball_y'] < 0.1)]['ball_x'].count()  # Adjust the threshold as needed
    return num_throw_ins

def calculate_num_shots(df):
    """
    Calculate the number of shots.
    
    Args:
    - df (DataFrame): DataFrame containing ball position data.
    
    Returns:
    - num_shots (int): Number of shots.
    """
    shots = np.diff(df['ball_y']) > 0.1  # Adjust the threshold as needed
    num_shots = np.sum(shots)
    return num_shots

def calculate_num_goals(df):
    """
    Calculate the number of goals.
    
    Args:
    - df (DataFrame): DataFrame containing ball position data.
    
    Returns:
    - num_goals (int): Number of goals.
    """
    ball_y_diff = np.diff(df['ball_y'])  # Calculate the difference between consecutive ball_y values
    goals = (df['ball_y'].iloc[1:] > 0.8) & (ball_y_diff > 0.1)  # Adjust the threshold as needed
    num_goals = goals.sum()
    return num_goals

def calculate_player_metrics(df, output_file):
    """
    Calculate various metrics for each player and write the results to a CSV file.
    
    Args:
    - df (DataFrame): DataFrame containing player position and event data.
    - output_file (str): Path to the output CSV file.
    """
    player_metrics = []

    for column in df.columns:
        if column.endswith('_x'):
            player = column.split('_')[0]
            x_values = df[column].values
            y_column = column.replace('_x', '_y')
            y_values = df[y_column].values
            time_column = column.replace('_x', '_t')
            time_values = df[time_column].values

            total_distance = calculate_distance(x_values, y_values)
            top_speed = calculate_top_speed(x_values, y_values, time_values)
            num_touches = calculate_num_touches(df, player)
            num_passes = calculate_num_passes(df, player)
            num_corner_kicks = calculate_num_corner_kicks(df)
            num_throw_ins = calculate_num_throw_ins(df)
            num_shots = calculate_num_shots(df)
            num_goals = calculate_num_goals(df)

            player_metrics.append({
                'Player': player,
                'Total Distance': total_distance,
                'Top Speed': top_speed,
                'Touches': num_touches,
                'Passes': num_passes,
                'Corner Kicks': num_corner_kicks,
                'Throw-ins': num_throw_ins,
                'Shots': num_shots,
                'Goals': num_goals
            })

    player_metrics_df = pd.DataFrame(player_metrics)
    player_metrics_df.to_csv(output_file, index=False)
    print("Player metrics saved to:", output_file)

# Run the functions on the provided DataFrame and save the results
calculate_player_metrics(df_copy, 'player_metrics.csv')



Player metrics saved to: player_metrics.csv
